# sumaryzacja tekstu

In [32]:
from google.colab import drive


drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [33]:
!nvidia-smi

Sun Nov 16 13:21:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P0             26W /   70W |    1510MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [34]:
# 1. Force uninstall both to clear the conflict
!pip uninstall -y unsloth unsloth_zoo

# 2. Install Unsloth Zoo DIRECTLY from Git (to get 'tiled_mlp')
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# 3. Install Unsloth main library from Git
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2025.11.3
Uninstalling unsloth-2025.11.3:
  Successfully uninstalled unsloth-2025.11.3
Found existing installation: unsloth_zoo 2025.11.4
Uninstalling unsloth_zoo-2025.11.4:
  Successfully uninstalled unsloth_zoo-2025.11.4
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-install-4l4l6c3u/unsloth-zoo_3b38f3da3bef4c269ccefed9188fcd21
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-4l4l6c3u/unsloth-zoo_3b38f3da3bef4c269ccefed9188fcd21
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit b60fd19e4e1c27014553f7194669d66cdb96db2c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 18.4 MB/s eta 0:00:00
  Created wheel for unsloth_zoo: filename=unsloth_zoo-2025.11.4-py3-none-any.whl size=281896 sha256=2ba1b593b2187e74d

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-oz8kwcwz/unsloth_2596a9c35854424baf6a84e25d7a25da
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-oz8kwcwz/unsloth_2596a9c35854424baf6a84e25d7a25da
  Resolved https://github.com/unslothai/unsloth.git to commit 38bdbed0bf65230bf076fd7e59583006348bde47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.11.3-py3-none-any.whl size=357217 sha256=562260bd4440a7946dbfdc7128f27bc9c9af0ef22a910c1e80a17722d56e1008
  Stored in directory: /tmp/pip-ephem-wheel-cache-lg7_q1i0/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


In [35]:
!pip install "datasets<3.0.0"


  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.3.0
    Uninstalling datasets-4.3.0:
      Successfully uninstalled datasets-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.24.0 requires datasets>=3.0.0, but you have datasets 2.21.0 which is incompatible.
unsloth-zoo 2025.11.4 requires datasets!=4.0.*,!=4.1.0,<4.4.0,>=3.4.1, but you have datasets 2.21.0 which is incompatible.


In [36]:
!pip install bert-extractive-summarizer

In [37]:
!pip install pytorch-ignite
!pip install torchmetrics

## importowanie datesetu

In [38]:
from datasets import load_dataset


ds = load_dataset("xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 750)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

In [39]:
print(ds)
print(shots)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1727
    })
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2
})


In [40]:
from summarizer import Summarizer
model = Summarizer()

The following layers were not sharded: encoder.layer.*.attention.self.key.weight, embeddings.token_type_embeddings.weight, embeddings.position_embeddings.weight, embeddings.LayerNorm.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value.bias, pooler.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.self.value.weight, encoder.layer.*.output.dense.bias, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.key.bias, encoder.layer.*.attention.output.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.query.bias, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight, pooler.dense.bias, encoder.layer.*.attention.self.query.weight


In [41]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 2], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.6, 'Rouge-L-R': 0.5, 'Rouge-L-F': 0.5, 'Rouge-2-P': 0.5, 'Rouge-2-R': 0.4, 'Rouge-2-F': 0.4}


In [42]:
from pprint import pprint
from torchmetrics.text.bert import BERTScore
preds = """
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.
"That may not be true but it is perhaps my perspective over the last few days.
"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"
Meanwhile, a flood alert remains in place across the Borders because of the constant rain.
Peebles was badly hit by problems, sparking calls to introduce more defences in the area.
Scottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.
The Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
He said it was important to get the flood protection plan right but backed calls to speed up the process.
"I was quite taken aback by the amount of damage that has been done," he said.
"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."
He said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.
Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.
"""
result = model(preds)
# print(result)
target ="""
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.
"""
print(f'target: {target}')
print(f'result: {result}')
bertscore = BERTScore(model_name_or_path="microsoft/deberta-v3-large", lang='en')
pprint(bertscore([result], [target]))

target: 
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.

result: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. First Minister Nicola Sturgeon visited the area to inspect the damage. "That may not be true but it is perhaps my perspective over the last few days. He said it was important to get the flood protection plan right but backed calls to speed up the process.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.

{'f1': tensor(0.7959), 'precision': tensor(0.7827), 'recall': tensor(0.8095)}


In [43]:
num_workers = 1
batch_size = 1

In [44]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"]
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 1727
})


In [45]:
def bert_score_loss(batch, model, bert: bool, tokenizer: any, prompt = None, promptsuff=None):
    if bert:
        predictions = [model(doc, ratio=0.1) for doc in batch['document']]
    else:
        docs = [doc for doc in batch['document']]
        prompted  = [f'{prompt}{doc}{promptsuff}'for doc in batch['document']]
        inputs = tokenizer(
        prompted,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096
    )
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        do_sample=True,
        top_k=0,
        temperature=0.5,
    )
        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]
        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )
    print(f"Doc: {batch['document'][0][:100]}")
    print(f"Prediction: {predictions[0]}")
    print(f"Target: {batch['summary'][0]}")
    predictions = [pred if pred.strip() else "." for pred in predictions]
    for i, pred in enumerate(predictions):
        if pred.strip() == "":
            print("EMPTY PREDICTION")
            print(batch['document'][i])
    return bertscore(predictions, batch["summary"])

In [46]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [47]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [48]:
def calc_loss_loader(data_loader, model, device, num_batches: int, bert: bool, tokenizer: any,prompt = None, promptsuff = None) -> float:
    total_loss = {'precision': [], 'recall': [], 'f1': []}
    for i, batch in enumerate(data_loader):
        if i >= num_batches:
            break
        # batch = batch_to_device(batch, device)
        loss = bert_score_loss(batch, model, bert, tokenizer, prompt, promptsuff)
        total_loss['precision'].append(loss['precision'].unsqueeze(0))
        total_loss['recall'].append(loss['recall'].unsqueeze(0))
        total_loss['f1'].append(loss['f1'].unsqueeze(0))
    total_loss['precision'] = torch.cat(total_loss['precision'])
    total_loss['recall'] = torch.cat(total_loss['recall'])
    total_loss['f1'] = torch.cat(total_loss['f1'])
    print(total_loss)
    return (sum(total_loss['precision'])/ len(total_loss['precision']),sum(total_loss['recall'])/ len(total_loss['recall']),sum(total_loss['f1'])/ len(total_loss['f1']))

In [49]:
def evaluate_model(model: nn.Module, loader: DataLoader,
                    device, eval_iter: int, bert: bool, tokenizer: any, prompt="", promptsuff ="") -> tuple[float, float]:
    if not bert:
      model.eval()
    with torch.no_grad():
        loss = calc_loss_loader(loader, model, device, num_batches=eval_iter, bert=bert, tokenizer=tokenizer, prompt=prompt, promptsuff=promptsuff)
    return loss

In [50]:
from torch import nn
loss = evaluate_model(model, test_loader, device='cuda', eval_iter=1, bert=True, tokenizer=None)
print(loss)



Doc: The blast happened in the early hours of Monday at the Weston Road branch in Long Ashton, near Brist
Prediction: The blast happened in the early hours of Monday at the Weston Road branch in Long Ashton, near Bristol. Owners David Andrews and Andrew Crabb said they hope to reopen "as soon as possible" after assessing the damage.
Target: Two more people have been arrested over an explosion at a post office by thieves attempting to steal a cash machine from its front.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.

{'precision': tensor([0.7712]), 'recall': tensor([0.8025]), 'f1': tensor([0.7865])}
(tensor(0.7712), tensor(0.8025), tensor(0.7865))


In [51]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id
model2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

The following layers were not sharded: transformer.h.*.mlp.c_proj.weight, transformer.h.*.attn.c_attn.weight, transformer.h.*.attn.c_attn.bias, transformer.h.*.mlp.c_proj.bias, transformer.wpe.weight, lm_head.weight, transformer.ln_f.weight, transformer.h.*.attn.c_proj.bias, transformer.wte.weight, transformer.h.*.mlp.c_fc.weight, transformer.h.*.mlp.c_fc.bias, transformer.ln_f.bias, transformer.h.*.ln_*.bias, transformer.h.*.attn.c_proj.weight, transformer.h.*.ln_*.weight


In [52]:
summary = "\n###Summary\n"

In [53]:
def generate_prompt(tokenizer, size):
    prompt = ""
    shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
    for batch in shots_loader:
        for i in range(len(batch['document'])):
            prompt += f'{i+1}. {batch['document'][i]}\n###Summary\n{batch['summary'][i]}'
            prompt += tokenizer.eos_token
    return prompt

prompt = generate_prompt(tokenizer, 3)
print(prompt)

1. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez levelled from close range.
World champions Germany moved above Chile with a 3-1 win over Cameroon and will play Mexico on Thursday (19:00).
RB Leipzig forward Timo Werner scored twice, with Hoffenheim midfielder Kerem Demirbay also on target for the experimental German squad competing in Russia.
Porto forward Vincent Aboubakar netted for Cameroon, who finished bottom of the group, with a draw from their match against Australia their only point.
Group A:
Group B:
###Summary
Chile qualified behind Germany from Group B to reach the Confederations Cup semi-finals after drawing 1-1 with Australia in their final group game.<|endoftext|>2. Several fire appliances were called to the scene at Carnsalloch House to tackle the blaze.
One witness reported damage to the main part of the house at Kirkton, with floors and the 

In [54]:
print(prompt)
print(evaluate_model(model2, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer, prompt = prompt, promptsuff=summary))


1. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez levelled from close range.
World champions Germany moved above Chile with a 3-1 win over Cameroon and will play Mexico on Thursday (19:00).
RB Leipzig forward Timo Werner scored twice, with Hoffenheim midfielder Kerem Demirbay also on target for the experimental German squad competing in Russia.
Porto forward Vincent Aboubakar netted for Cameroon, who finished bottom of the group, with a draw from their match against Australia their only point.
Group A:
Group B:
###Summary
Chile qualified behind Germany from Group B to reach the Confederations Cup semi-finals after drawing 1-1 with Australia in their final group game.<|endoftext|>2. Several fire appliances were called to the scene at Carnsalloch House to tackle the blaze.
One witness reported damage to the main part of the house at Kirkton, with floors and the 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: The 79-year-old West Ham chairman was sent a tweet by a prankster, asking to spread awareness of a "
Prediction: The West Ham chairman was sent a tweet by a prankster, asking to spread awareness of a "missing" Hammers fan
A Twitter user found a picture of a West Ham supporter with the caption: "West Ham chairman, Michail Antonio is
Target: David Gold thought he was doing a good deed, but instead was found guilty of not recognising one of his own players.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.7519]), 'recall': tensor([0.7960]), 'f1': tensor([0.7733])}
(tensor(0.7519), tensor(0.7960), tensor(0.7733))


In [55]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer3 = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer3.padding_side = 'left'
if tokenizer3.pad_token is None:
    tokenizer3.pad_token = tokenizer3.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer3.pad_token_id = tokenizer3.eos_token_id
model3 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")




tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: model.embed_tokens.weight, model.layers.*.input_layernorm.weight, lm_head.weight, model.norm.weight, model.layers.*.self_attn.v_proj.bias, model.layers.*.self_attn.k_proj.weight, model.layers.*.post_attention_layernorm.weight, model.layers.*.self_attn.q_proj.bias


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [56]:
prompt3 = generate_prompt(tokenizer3, 0)
print(evaluate_model(model3, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer3, prompt=prompt3, promptsuff = summary))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: The incident happened in December when the 18-year-old man suffered what was described at the time a
Prediction: A man has been arrested on suspicion of assaulting a member of the public in Dumfries.
He was arrested in a town centre car park following a dispute between two men.
The man, described as being in his early 20s, was
Target: Police investigating an "unprovoked serious assault" on a teenager outside Stonehaven railway station have released CCTV images of a woman and man they want to trace.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.8012]), 'recall': tensor([0.8100]), 'f1': tensor([0.8056])}
(tensor(0.8012), tensor(0.8100), tensor(0.8056))


In [57]:
prompt4 = "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? "
print(
    evaluate_model(
        model3,
        test_loader,
        device="cuda",
        eval_iter=1,
        bert=False,
        tokenizer=tokenizer3,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: A partial eclipse was visible in Australia, parts of South East Asia and the Pacific.
Prediction:  It covered the eastern part of the country, but was not visible in the northern parts of the country.

This text is about a partial eclipse happening in Australia, parts of Southeast Asia and the Pacific. The eclipse covered the eastern part of the country,
Target: People across Indonesia and the Pacific have witnessed a total solar eclipse, with some parts of Indonesia in total darkness for up to three minutes.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.7638]), 'recall': tensor([0.7998]), 'f1': tensor([0.7813])}
(tensor(0.7638), tensor(0.7998), tensor(0.7813))


In [58]:
print(
    evaluate_model(
        model2,
        test_loader,
        device="cuda",
        eval_iter=2,
        bert=False,
        tokenizer=tokenizer,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: The 27-year-old represented England Under-19s and scored 72 on his Glamorgan debut in 2006.
Wright, 
Prediction: 
"It is a loss for England but a great day for our young players and for our fans," he added.
Target: Glamorgan batsman Ben Wright will retire at the end of the season to pursue a career with a metal manufacturer.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: The 24-year-old Spaniard was with East Fife last season and impressed during a short trial at Dens P
Prediction: 
He was part of the squad that won the League Cup in the summer of 2014, helping them reach the Europa League final in the process.
The 29-year-old was also involved in the domestic competition for the first time since his loan
Target: Dundee have signed defender Julen Etxabeguren Leanizbarrutia on a three-year deal.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.7804, 0.7444]), 'recall': tensor([0.8013, 0.7399]), 'f1': tensor([0.7907, 0.7422])}
(tensor(0.7624), tensor(0.7706), tensor(0.7664))


In [59]:
import torch

print(torch.cuda.is_available())

True


In [60]:
# !pip install -U bitsandbytes

In [61]:
from unsloth import FastLanguageModel

ft_model, ft_tokenzier = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B", max_seq_length=4096, load_in_4bit=False
)

/tmp/ipython-input-4133930992.py:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: model.embed_tokens.weight, model.layers.*.input_layernorm.weight, lm_head.weight, model.norm.weight, model.layers.*.self_attn.v_proj.bias, model.layers.*.self_attn.k_proj.weight, model.layers.*.post_attention_layernorm.weight, model.layers.*.self_attn.q_proj.bias


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Qwen/Qwen1.5-0.5B does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [62]:
ft_ds = ds


def merge_columns(example):
    example["text"] = (
        f"{example['document']}\n### Summary\n{example['summary']}"
    )
    return example

ft_ds = ft_ds.map(merge_columns)

Map:   0%|          | 0/32053 [00:00<?, ? examples/s]

Map:   0%|          | 0/1746 [00:00<?, ? examples/s]

Map:   0%|          | 0/1727 [00:00<?, ? examples/s]

In [66]:
from trl import SFTTrainer, SFTConfig
ft_model = FastLanguageModel.get_peft_model(
    ft_model,
    r=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=1234,
    use_rslora = False,
    loftq_config = None,

)
trainer = SFTTrainer(
    model=ft_model,
    train_dataset=ft_ds["train"],
    eval_dataset=ft_ds["validation"],
    tokenizer=ft_tokenzier,
    args=SFTConfig(
        max_seq_length=4096,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=1,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_8bit",
        seed=3407,
        eval_strategy="steps",
        eval_steps=200,
    ),
)
trainer.train()

Unsloth: Already have LoRA adapters! We shall skip this step.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/32053 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1746 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32,053 | Num Epochs = 1 | Total steps = 4,007
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 15,138,816 of 479,126,528 (3.16% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tom-p-smolen (tom-p-smolen-politechnika-warszawska) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss,Validation Loss
200,2.381800,2.497161
400,2.427800,2.472306
600,2.709400,2.456755
800,2.328100,2.447483
1000,2.568700,2.438116
1200,2.372000,2.430838
1400,2.657300,2.423373
1600,2.500400,2.417961
1800,2.243900,2.414217
2000,2.313400,2.410524


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=4007, training_loss=2.4089501952779897, metrics={'train_runtime': 5739.6193, 'train_samples_per_second': 5.585, 'train_steps_per_second': 0.698, 'total_flos': 9864199128557568.0, 'train_loss': 2.4089501952779897, 'epoch': 1.0})

In [67]:
evaluate_model(ft_model, test_loader, 'cuda', 50, bert=False, tokenizer = ft_tokenzier, promptsuff = "\n### Summary\n")

Doc: Martin made 38 appearances last season for Livi, scoring five goals, as they finished sixth, one pla
Prediction: Former Livingston midfielder John Martin has signed a new contract with the League One side. The 32-year-old played 14 games for the club last season, helping them win promotion to Scottish Championship.
Target: Raith Rovers have signed Martin Scott after the 28-year-old midfielder was released by Scottish Championship rivals Livingston.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


Doc: The USA captain has been undergoing medical evaluations since last month, his club Seattle Sounders 
Prediction: Seattle Sounders captain Clint Dempsey is set to be sidelined for at least six months after undergoing a hip operation. The club confirmed on Thursday that he has been sent off for the first time since 2009.
Target: Former Tottenham and Fulham forward Clint Dempsey has undergone a procedure to address an irregular heartbeat.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


Doc: The 17-year-old boy was arrested at an address in Netherton, West Midlands Police said, and remains 
Prediction: A man has been arrested after a teenager was stabbed in a street in Dudley. The man was taken to hospital with stab wounds to his chest and back. A 65-year-old man was attacked in the street. He was treated for stab wounds
Target: A teenager has been arrested on suspicion of attempted murder after a man was stabbed on his doorstep in the West Midlands on Tuesday.


...

The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


Doc: Sadiq Khan said Lord Toby Harris of Haringey would review the "capability, capacity and collaboratio
Prediction: The Cabinet Secretary for Communities and Local Government has said he will be looking at how London's police forces can better work together. Lord Harris will report on the review.
Target: The Mayor of London has appointed a security expert to assess how well the capital would cope with a major terror attack.


The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.8136, 0.7546, 0.8048, 0.7971, 0.7899, 0.8234, 0.8438, 0.8350, 0.7365,
        0.8132, 0.8011, 0.7930, 0.8167, 0.8004, 0.7875, 0.7602, 0.8060, 0.7802,
        0.7815, 0.8014, 0.8716, 0.8294, 0.8353, 0.8086, 0.7703, 0.8069, 0.8058,
        0.7794, 0.7995, 0.7988, 0.7803, 0.8094, 0.8131, 0.8104, 0.7428, 0.7830,
        0.7665, 0.7536, 0.7711, 0.7827, 0.7877, 0.8230, 0.6467, 0.8998, 0.7566,
        0.8228, 0.7727, 0.7365, 0.8165, 0.7944]), 'recall': tensor([0.8446, 0.8118, 0.8367, 0.8442, 0.7877, 0.8068, 0.8468, 0.8629, 0.7829,
        0.8671, 0.8228, 0.8753, 0.8528, 0.8211, 0.7991, 0.7388, 0.8021, 0.8029,
        0.8033, 0.8595, 0.8758, 0.8811, 0.8097, 0.8673, 0.7945, 0.8325, 0.8567,
        0.7965, 0.7822, 0.8136, 0.7843, 0.7824, 0.8623, 0.8619, 0.7858, 0.8122,
        0.8382, 0.7722, 0.7862, 0.7973, 0.8392, 0.8071, 0.7913, 0.9341, 0.7806,
        0.7691, 0.8065, 0.7892, 0.8408, 0.8240]), 'f1': tensor([0.8288, 0.7822, 0.8204, 0.8200, 0.7888, 0.8150, 0.8453, 0.8487

(tensor(0.7943), tensor(0.8209), tensor(0.8070))

In [68]:
torch.save(ft_model, 'ft')

AttributeError: 'Qwen2ForCausalLM' object has no attribute 'save'